In [7]:
import sage.matroids.matroid
from operator import xor
import functools

In [119]:
def iOrder(a,b,t=1):
    # a <_t b, return negative
    # a >_t b, return positive
    
    if a == b: return 0
    if xor((a < t),(b < t)):
        return -(a-b)
    return a-b

In [185]:
def galeHelper(I,order=1):
    I.sort()
    if order == 1: return I
    
    result=[]
    for i in range(len(I)):
        if I[i] >= order:
            result = I[i:] + I[:i]
            break
    if len(result) == 0: return I
    return result

In [177]:
def galeOrder(I,J,order=1):
    # non-comparable, return None
    # I <=_order J under Gale order, return negative
    # I >=_order J, return positive
    # I = J, return 0

    if len(I) == 0 or len(J) == 0:
        return 0
    
    t = order
    
    I = galeHelper(I, order)
    J = galeHelper(J, order)
    
    flag = iOrder(I[0],J[0],t)
    
    for j in range(len(I)):
        # compare I,J one by one
        current = iOrder(I[j],J[j],t)

        if current == 0:
            continue #doesn't change whatever we had
            
        if flag == 0:
            flag = current
            
        if flag != 0 and xor(flag < 0, current < 0):
            return None
        
        
    return flag

In [244]:
class DecoratedPermutation:
    def __init__(self, p, u=[], d=[]):
        self.p = Permutation(p)
        self.fp = self.p.fixed_points()
        if not (set(u+d) <= set(self.fp)):
            print("decorations are not fixed points!!!")
        self.u = u
        self.d = d
    def perm(self):
        return self.p
    def up(self):
        return self.u
    def down(self):
        return self.d
    def fixed_points(self):
        return self.fp

In [245]:
def dpToGrNeck(dp):
    n = len(dp.perm())
    grNeck = {}
    for r in range(1,n+1):
        current = []
        for i in (list(range(r,n+1))+list(range(1,r))):
            pi = dp.perm()
            piInv = dp.perm().inverse()
            if (iOrder(i,piInv(i),r) < 0) or (pi(i) == i and i in dp.down()):
                current.append(i)
        grNeck[r] = current
    return grNeck

def grNeckToDp(grNeck):
    n = len(grNeck)
    p = []
    u = []
    d = []
    for i in range(1,n):
        if grNeck[i+1] != grNeck[i]:
            j = set(grNeck[i+1]+grNeck[i]) - set(grNeck[i+1]).intersection(set(grNeck[i]))
            j = list(j - set({i}))[0]
            p.append(j)
        else:
            p.append(i)
            if i not in grNeck[i]:
                u.append(i)
            else:
                d.append(i)
                
    left = list(set(range(1,n+1)) - set(p))[0]
    p.append(left)
    if left == n:
        if left not in grNeck[n]:
            u.append(left)
        else:
            d.append(left)

    return DecoratedPermutation(p,u,d)

In [246]:
def grNeckToBases(grNeck):
    n = len(grNeck)
    k = len(grNeck[1])
    bases = []

    for B in Subsets(n,k).list():
        B = B.list()
        flag = 0
        for t in range(1,n+1):
            
            if galeOrder(grNeck[t], B, t) == None or galeOrder(grNeck[t], B, t) > 0:
                flag = 1
                break
        if flag == 0: bases.append(B)
    return bases

In [247]:
class Positroid(sage.matroids.matroid.Matroid):
    def __init__(self, n, dp=None, grNeck=None):
        self.E = list(range(1,n+1))
        self.n = n
        
        if dp != None:
            self.dp = dp
            self.grNeck = dpToGrNeck(dp)
        if grNeck != None:
            self.dp = grNeckToDp(grNeck)
            self.grNeck = grNeck
            
    def groundset(self):
        return self.E
    def _rank(self, X):
        M = Matroid(groundset = self.E, bases = grNeckToBases(self.grNeck))
        return M.rank(X)
        
    

In [248]:
for b in Positroid(6, dp=DecoratedPermutation([5,3,2,1,4,6],[],[6])).bases():
    print(list(b))

[1, 2, 4, 6]
[1, 2, 5, 6]
[1, 3, 4, 6]
[1, 3, 5, 6]
[2, 4, 5, 6]
[3, 4, 5, 6]


In [256]:
grNeck = dpToGrNeck(DecoratedPermutation([5,3,2,1,4,6],[],[6]))
dp = grNeckToDp(grNeck)
print(dp.perm(),dp.up(),dp.down())

[5, 3, 2, 1, 4, 6] [] [6]


In [253]:
for b in Positroid(6, grNeck=grNeck).bases():
    print(list(b))

[1, 2, 4, 6]
[1, 2, 5, 6]
[1, 3, 4, 6]
[1, 3, 5, 6]
[2, 4, 5, 6]
[3, 4, 5, 6]
